# 0. Package Imports and Functions

In [11]:
import os
import sys
import pandas as pd
import json
import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import deque

from morfeus import Sterimol, read_xyz
from morfeus.plotting import get_drawing_arrow
import pyvista as pv
# pv.set_jupyter_backend('ipyvtklink')
from pyvistaqt import BackgroundPlotter

from matplotlib.colors import hex2color
from morfeus.data import jmol_colors

import sterimol_functions as sf

from datetime import date
run_date = date.today().strftime("%b-%d-%Y")

# 1. Load File

In [12]:
file_name = 'pp000002.xyz'
file_directory = './xyz_files/'

# 2. Calculate Sterimol Values

In [15]:
atom_1 = 2  # 1-indexed
atom_2 = 63  # 1-indexed

elements, coordinates = read_xyz(os.path.join(file_directory, file_name))
sterimol = Sterimol(elements, coordinates, atom_1, atom_2)
print(f'Sterimol L: {sterimol.L_value:.3f}')
print(f'Sterimol B1: {sterimol.B_1_value:.3f}')
print(f'Sterimol B5: {sterimol.B_5_value:.3f}')

Sterimol L: 6.732
Sterimol B1: 3.804
Sterimol B5: 6.717


# 3. Optional: Visualize in 3D (opens in pyvista)

In [4]:
sf.plot_sterimol_3d(sterimol)

# 4. Determine Vector Coordinates

Note: Sterimol translates and rotates the molecule coordinates, which is used by PyVista. Visualization in other programs requires rotation of the sterimol vectors so they are relative to the original coordinates.

In [9]:
# Indices you used for Sterimol
idx_dummy = atom_1-1  # Convert to 0-based index
idx_attached = atom_2-1  # Convert to 0-based index
idx_sub = 0  # <-- set this to a third atom index, not colinear with the first two

color = (96/255, 113/255, 163/255)  # RGB color (divide by 255 for PyMOL compatibility)
color = tuple(round(c, 3) for c in color)

""" User input not required below this line. """

# Get coordinates in original frame
A_orig = coordinates[idx_dummy]
B_orig = coordinates[idx_attached]
C_orig = coordinates[idx_sub]

# Get coordinates in Sterimol frame
A_rot = sterimol._dummy_atom.coordinates
B_rot = sterimol._attached_atom.coordinates
C_rot = sterimol._atoms[idx_sub].coordinates

axes_orig = sf.build_axes(A_orig, B_orig, C_orig)
axes_rot = sf.build_axes(A_rot, B_rot, C_rot)

# Rotation matrix from Sterimol frame to original frame
R = axes_orig @ np.linalg.inv(axes_rot)

# Transform Sterimol vectors back to original frame
L_vec_orig = R @ sterimol.L
B1_vec_orig = R @ sterimol.B_1
B5_vec_orig = R @ sterimol.B_5

# Arrow start and end points in original frame
start_L = np.round(A_orig, 3).tolist()
end_L   = np.round(A_orig + L_vec_orig, 3).tolist()

start_B = np.round(B_orig, 3).tolist()
end_B1  = np.round(B_orig + B1_vec_orig, 3).tolist()
end_B5  = np.round(B_orig + B5_vec_orig, 3).tolist()

print("Sterimol L:\n", f'draw_arrow({np.round(start_L,3).tolist()}, {np.round(end_L,3).tolist()}, color={color}, name="L")')
print("Sterimol B1:\n", f'draw_arrow({np.round(start_B,3).tolist()}, {np.round(end_B1,3).tolist()}, color={color}, name="B1")')
print("Sterimol B5:\n", f'draw_arrow({np.round(start_B,3).tolist()}, {np.round(end_B5,3).tolist()}, color={color}, name="B5")')

Sterimol L:
 draw_arrow([-1.609, 0.237, 0.003], [2.777, -4.313, -0.389], color=(0.376, 0.443, 0.639), name="L")
Sterimol B1:
 draw_arrow([-0.064, -1.366, -0.135], [-2.16, -3.582, 2.137], color=(0.376, 0.443, 0.639), name="B1")
Sterimol B5:
 draw_arrow([-0.064, -1.366, -0.135], [3.574, 2.495, -4.255], color=(0.376, 0.443, 0.639), name="B5")


# 4. Scale Coordinates for PyMOL Visualization

Note: this is NOT the correct sterimol length (van der waals radii is being ignored), this is ONLY to plot a pretty line matching the size of atoms used in PyMOL. Some adjustment of length may be required.

In [8]:
color = (96/255, 113/255, 163/255)  # RGB color (divide by 255 for PyMOL compatibility)
color = tuple(round(c, 3) for c in color)

start_L, end_L = sf.shorten_vector(A_orig, L_vec_orig, 1.3)  # Shorten by 1.3 Å
start_B1, end_B1 = sf.shorten_vector(B_orig, B1_vec_orig, 1.0)  # Shorten by 1.0 Å
start_B5, end_B5 = sf.shorten_vector(B_orig, B5_vec_orig, 0.8)  # Shorten by 0.8 Å

print("Sterimol L:\n", f'draw_arrow({np.round(start_L,3).tolist()}, {np.round(end_L,3).tolist()}, color={color}, name="L")')
print("Sterimol B1:\n", f'draw_arrow({np.round(start_B1,3).tolist()}, {np.round(end_B1,3).tolist()}, color={color}, name="B1")')
print("Sterimol B5:\n", f'draw_arrow({np.round(start_B5,3).tolist()}, {np.round(end_B5,3).tolist()}, color={color}, name="B5")')

Sterimol L:
 draw_arrow([-1.609, 0.237, 0.003], [1.877, -3.379, -0.309], color=(0.376, 0.443, 0.639), name="L")
Sterimol B1:
 draw_arrow([-0.064, -1.366, -0.135], [-1.609, -3.0, 1.54], color=(0.376, 0.443, 0.639), name="B1")
Sterimol B5:
 draw_arrow([-0.064, -1.366, -0.135], [3.141, 2.035, -3.764], color=(0.376, 0.443, 0.639), name="B5")
